## Clean the data and prepare it for fitting classificaiton / regression models

Column definitions (from project challenge description)

    DATOP - Date of flight
    FLTID - Flight number
    DEPSTN - Departure point
    ARRSTN - Arrival point
    STD - Scheduled Time departure
    STA - Scheduled Time arrival
    STATUS - Flight status
    ETD - Expected Time departure
    ETA - Expected Time arrival
    ATD - Actual Time of Departure
    ATA - Actual Time of arrival
    DELAY1 - Delay code 1
    DUR1 - delay time 1
    DELAY2 - Delay code 2
    DUR2 - delay time 2
    DELAY3 - Delay code 3
    DUR3 - delay time 3
    DELAY4 - Delay code 4
    DUR4 - delay time 4
    AC - Aircraft Code


In [20]:
#import libraries
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pickle

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

#import supporting functions used for cleaning
from supporting_functions import check_duplicates

In [21]:
#load data
df = pd.read_csv('data/Train.csv',sep=',')

In [22]:
#user-defined target for classification
target_column = 'target'

## Main function for cleaning

In [23]:
def clean_data(df,target_column):
    ### main preprocessing function:

    ## done on train and test set together

    #Log transform the target
    df[target_column] = df.target.apply(lambda x: np.log1p(x) if x > 0 else 0)
    
    #clean up column names
    df.columns = [col.lower() for col in df.columns]

    #check and report on duplicate data
    df = check_duplicates(df)

    ## some feature engineering

    #split date into day, month, year, convert each three to number
    df['datop'] = pd.to_datetime(df['datop'])
    #df['year'] = df['datop'].dt.year #we cannot use year because it probably won't generalize to future years
    df['month'] = df['datop'].dt.month
    df['day'] = df['datop'].dt.day
    
    df.columns = [col.replace(' ', '_') for col in df.columns]

    #combine STD and STA into one column with scheduled flight duration	
    df['std'] = pd.to_datetime(df['std'])
    df['sta'] = pd.to_datetime(df['sta'],format='%Y-%m-%d %H.%M.%S')
    df['flight_time'] = (df['sta'] - df['std']).dt.total_seconds() / 60 # Calculate the flight time in minutes

    #save departure and arrival hour
    df['std_hour'] = df['std'].dt.hour
    df['sta_hour'] = df['sta'].dt.hour

    # extract the letter part from 'fltid' column and create a new column 'flight_code'
    df['flight_code'] = df['fltid'].str.split(' ').str[0]

    #drop column names that we don't need
    #df = df.drop(['id', 'fltid','std', 'sta', 'datop','ac'],axis=1)
    df = df.drop(['id', 'fltid','std', 'sta', 'datop'],axis=1)
    #df = df.drop(['arrstn', 'depstn', 'id', 'fltid','std', 'sta', 'datop'],axis=1)

    #dummy code
    #columns_to_dummycode = ['arrstn', 'depstn', 'status', 'flight_code'] #dummy-code arrstn, depstn, and status, ac
    #columns_to_dummycode = ['arrstn', 'depstn', 'status', 'ac', 'flight_code'] #dummy-code arrstn, depstn, and status, ac
    #columns_to_dummycode = ['status', 'ac', 'flight_code'] #dummy-code arrstn, depstn, and status, ac
    #df = one_hot(df,columns_to_dummycode)

    cat_columns = ['arrstn', 'depstn', 'status', 'ac', 'flight_code'] #dummy-code arrstn, depstn, and status, ac
    df[cat_columns] = df[cat_columns].astype('category')

    df.columns = [col.lower() for col in df.columns]   

    

    #run train-test split
    #note: X still contains the y-variable in the 'target' column, this is because it 
    #easier to remove rows / apply cleaning steps without having to do it separately 
    #for the target data vector.
    X = df
    X_train, X_test, _, _ = train_test_split(X, df[target_column], random_state=0,test_size=0.2) 

    X_train.reset_index(drop=True, inplace=True)
    X_test.reset_index(drop=True, inplace=True)


    ## done separately for train and test

    #define / identify columns for range normalization
    columns_to_scale = df.select_dtypes(include='number').columns.drop(target_column) #identify the numeric columns
  
    #remove outliers from the training set
    outlier_threshold = X_train['flight_time'].median()+(X_train['flight_time'].std()*3)
    print(str(np.sum(X_train['flight_time'] > outlier_threshold)) + " outliers detected")
    X_train = X_train[X_train['flight_time'] < outlier_threshold]
    X_train.reset_index(drop=True, inplace=True)       

    #range normalization
    scaler = MinMaxScaler()
    #scaler = StandardScaler()
    scaler.set_output(transform="pandas")

    X_train_scaled = scaler.fit_transform(X_train[columns_to_scale])
    X_test_scaled  = scaler.transform(X_test[columns_to_scale])
    X_train = pd.concat([X_train_scaled, X_train.drop(columns_to_scale,axis=1)], axis=1)
    X_test  = pd.concat([X_test_scaled, X_test.drop(columns_to_scale,axis=1)], axis=1)

    #separate the target
    y_train = X_train.pop(target_column)
    y_test  = X_test.pop(target_column)

    return X_train, X_test, y_train, y_test, scaler

## Clean the data and make features

In [24]:
X_train, X_test, y_train, y_test, scaler = clean_data(df,target_column)

No duplicates found
38 outliers detected


## Select features and check the model

In [25]:
X_train.shape

(86228, 10)

In [26]:
X_train.head()

,month,day,flight_time,std_hour,sta_hour,depstn,arrstn,status,ac,flight_code
0,0.000000,0.666667,0.003077,0.652174,0.695652,DJE,TUN,SCH,TU CR9ISA,UG
1,0.545455,0.766667,0.005846,0.304348,0.391304,TUN,ORN,ATA,TU 32AIMM,TU
2,0.909091,0.800000,0.002154,0.913043,0.956522,MIR,DJE,ATA,TU 736IOP,TU
3,0.636364,0.033333,0.008308,0.260870,0.347826,ORY,TUN,ATA,TU 32AIML,TU
4,0.909091,0.166667,0.010462,0.608696,0.739130,TUN,CAI,ATA,TU 32AIMF,TU


In [27]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86228 entries, 0 to 86227
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   month        86228 non-null  float64 
 1   day          86228 non-null  float64 
 2   flight_time  86228 non-null  float64 
 3   std_hour     86228 non-null  float64 
 4   sta_hour     86228 non-null  float64 
 5   depstn       86228 non-null  category
 6   arrstn       86228 non-null  category
 7   status       86228 non-null  category
 8   ac           86228 non-null  category
 9   flight_code  86228 non-null  category
dtypes: category(5), float64(5)
memory usage: 3.9 MB


In [28]:
#in case we want to do feature filtering, we can un-comment the code below

discarded_features = []
#select features with filtering
# thresh = 0.95 #remove features that correlate above this threshold
#X_train, X_test, discarded_features = filter_features(X_train,X_test,thresh)

#plot correlation between regressors
# sns.heatmap(X_train.corr(),vmin=-1,vmax=1,cmap='seismic')
# plt.title('Correlations in design matrix')

## Save everything to file

In [29]:
data_to_save = [X_train, X_test, y_train, y_test, discarded_features, scaler]

# save data to file so we don't have to run it again
with open('data/clean_data.pkl','wb') as f:
    pickle.dump(data_to_save,f)    